In [2]:
#Qiskit Machine Learning - Option Pricing with qGANs¶
#qGAN can facilitate the pricing of a European call option
#a qGAN can be trained such that a quantum circuit models the spot price of an asset underlying a European call option. 
#The resulting model can then be integrated into a Quantum Amplitude Estimation based algorithm to evaluate the expected payoff 
#Paper: Quantum Generative Adversarial Networks for Learning and Loading Random Distributions. Zoufal, Lucchi, Woerner. 2019.

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua.components.uncertainty_models import (UniformDistribution,
                                                       UnivariateVariationalDistribution)
from qiskit.aqua.algorithms import QGAN
from qiskit.aqua import aqua_globals, QuantumInstance, MissingOptionalLibraryError
from qiskit.aqua.components.initial_states import Custom
from qiskit.aqua.components.optimizers import CG, COBYLA
from qiskit.aqua.components.neural_networks import NumPyDiscriminator, PyTorchDiscriminator
from qiskit import BasicAer

In [35]:
class TestQGAN():

    def setUp(self):
        self.seed = 7
        aqua_globals.random_seed = self.seed
        # Number training data samples
        n_v = 5000
        # Load data samples from log-normal distribution with mean=1 and standard deviation=1
        m_u = 1
        sigma = 1
        self._real_data = aqua_globals.random.lognormal(mean=m_u, sigma=sigma, size=n_v)
        # Set upper and lower data values as list of k
        # min/max data values [[min_0,max_0],...,[min_k-1,max_k-1]]
        self._bounds = [0., 3.]
        # Set number of qubits per data dimension as list of k qubit values[#q_0,...,#q_k-1]
        num_qubits = [2]
        # Batch size
        batch_size = 100
        # Set number of training epochs
        # num_epochs = 10
        num_epochs = 5

        # Initialize qGAN
        self.qgan = QGAN(self._real_data,
                         self._bounds,
                         num_qubits,
                         batch_size,
                         num_epochs,
                         snapshot_dir=None)
        self.qgan.seed = 7
        # Set quantum instance to run the quantum generator
        self.qi_statevector = QuantumInstance(backend=BasicAer.get_backend('statevector_simulator'),
                                              seed_simulator=2,
                                              seed_transpiler=2)
        self.qi_qasm = QuantumInstance(backend=BasicAer.get_backend('qasm_simulator'),
                                       shots=1000,
                                       seed_simulator=2,
                                       seed_transpiler=2)
        # Set entangler map
        entangler_map = [[0, 1]]

        # Set an initial state for the generator circuit
        init_dist = UniformDistribution(sum(num_qubits), low=self._bounds[0], high=self._bounds[1])
        q = QuantumRegister(sum(num_qubits), name='q')
        qc = QuantumCircuit(q)
        init_dist.build(qc, q)
        init_distribution = Custom(num_qubits=sum(num_qubits), circuit=qc)

        # Set generator's initial parameters
        init_params = aqua_globals.random.random(2 * sum(num_qubits)) * 2 * 1e-2

        # Set variational form
        var_form = RealAmplitudes(sum(num_qubits), reps=1, initial_state=init_distribution,
                                  entanglement=entangler_map)
        self.generator_circuit = var_form
        self.generator_factory = UnivariateVariationalDistribution(sum(num_qubits), var_form,
                                                                   init_params,
                                                                   low=self._bounds[0],
                                                                   high=self._bounds[1])

    def test_sample_generation(self, circuit_type):
        if circuit_type == 'factory':
            self.qgan.set_generator(generator_circuit=self.generator_factory)
        else:
            self.qgan.set_generator(generator_circuit=self.generator_circuit)

        _, weights_statevector = self.qgan._generator.get_output(self.qi_statevector, shots=100)
        samples_qasm, weights_qasm = self.qgan._generator.get_output(self.qi_qasm, shots=100)
        samples_qasm, weights_qasm = zip(*sorted(zip(samples_qasm, weights_qasm)))
        for i, weight_q in enumerate(weights_qasm):
            print(i)
            print(weight_q)
            print(weights_statevector[i])

    def test_qgan_training_cg(self):
        optimizer = CG(maxiter=1)
        self.qgan.set_generator(generator_circuit=self.generator_circuit,
                                generator_optimizer=optimizer)
        trained_statevector = self.qgan.run(self.qi_statevector)
        trained_qasm = self.qgan.run(self.qi_qasm)
        print(trained_qasm['rel_entr'])
        print(trained_statevector['rel_entr'])

    def test_qgan_training_cobyla(self):
        optimizer = COBYLA(maxiter=1)
        self.qgan.set_generator(generator_circuit=self.generator_circuit,
                                generator_optimizer=optimizer)
        trained_statevector = self.qgan.run(self.qi_statevector)
        trained_qasm = self.qgan.run(self.qi_qasm)
        print(trained_qasm['rel_entr'])
        print(trained_statevector['rel_entr'])

    def test_qgan_training(self):
        self.qgan.set_generator(generator_circuit=self.generator_circuit)

        trained_statevector = self.qgan.run(self.qi_statevector)
        trained_qasm = self.qgan.run(self.qi_qasm)
        print(trained_qasm['rel_entr'])
        print(trained_statevector['rel_entr'])

    def test_qgan_training_run_algo_torch(self):
        try:
            # Set number of qubits per data dimension as list of k qubit values[#q_0,...,#q_k-1]
            num_qubits = [2]
            # Batch size
            batch_size = 100
            # Set number of training epochs
            num_epochs = 5
            _qgan = QGAN(self._real_data,
                         self._bounds,
                         num_qubits,
                         batch_size,
                         num_epochs,
                         discriminator=PyTorchDiscriminator(n_features=len(num_qubits)),
                         snapshot_dir=None)
            _qgan.seed = self.seed
            _qgan.set_generator()
            trained_statevector = _qgan.run(QuantumInstance(
                BasicAer.get_backend('statevector_simulator'),
                seed_simulator=aqua_globals.random_seed,
                seed_transpiler=aqua_globals.random_seed))
            trained_qasm = _qgan.run(QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                                     seed_simulator=aqua_globals.random_seed,
                                                     seed_transpiler=aqua_globals.random_seed))
            print(trained_qasm['rel_entr'])
            print(trained_statevector['rel_entr'])
        except MissingOptionalLibraryError:
            print('pytorch not installed, skipping test')

    def test_qgan_training_run_algo_numpy(self):
        # Set number of qubits per data dimension as list of k qubit values[#q_0,...,#q_k-1]
        num_qubits = [2]
        # Batch size
        batch_size = 100
        # Set number of training epochs
        num_epochs = 5
        _qgan = QGAN(self._real_data,
                     self._bounds,
                     num_qubits,
                     batch_size,
                     num_epochs,
                     discriminator=NumPyDiscriminator(n_features=len(num_qubits)),
                     snapshot_dir=None)
        _qgan.seed = self.seed
        _qgan.set_generator()
        trained_statevector = _qgan.run(
            QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                            seed_simulator=aqua_globals.random_seed,
                            seed_transpiler=aqua_globals.random_seed))
        trained_qasm = _qgan.run(QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                                 seed_simulator=aqua_globals.random_seed,
                                                 seed_transpiler=aqua_globals.random_seed))
        print(trained_qasm['rel_entr'])
        print(trained_statevector['rel_entr'])



In [36]:
testQGAN = TestQGAN()
testQGAN.setUp()

E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\aqua\components\uncertainty_models\uncertainty_model.py:33: DeprecationWarning: The UniformDistribution is deprecated as of Aqua 0.8.0 and will be removed no earlier than 3 months after the release date. You can use qiskit.circuit.library.UniformDistribution instead.
  super().__init__(num_target_qubits)
E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\aqua\components\uncertainty_models\uncertainty_model.py:33: DeprecationWarning: The UnivariateVariationalDistribution is deprecated as of Aqua 0.8.0 and will be removed no earlier than 3 months after the release date. You can use a parameterized qiskit.QuantumCircuit instead.
  super().__init__(num_target_qubits)


In [18]:
testQGAN.test_qgan_training()

0.1836
0.1836


In [25]:
testQGAN.test_sample_generation('factory')

0
0.24
0.24005186277854557
1
0.3
0.25178990773177046
2
0.28
0.25689384835765766
3
0.18
0.2512643811320263


E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\aqua\algorithms\distribution_learners\qgan.py:209: DeprecationWarning: Passing a UnivariateVariationalDistribution or MultivariateVariationalDistribution is as ``generator_circuit`` is deprecated as of Aqua 0.8.0 and the support will be removed no earlier than 3 months after the release data. You should pass as QuantumCircuit instead.
  self._generator = QuantumGenerator(self._bounds, self._num_qubits,


In [28]:
testQGAN.test_qgan_training_cg()

0.2924
0.2924


In [33]:
testQGAN.test_qgan_training_cobyla()

0.1821
0.1821


In [34]:
testQGAN.test_qgan_training_run_algo_torch()

pytorch not installed, skipping test


In [37]:
testQGAN.test_qgan_training_run_algo_numpy()

0.2138
0.2138
